This notebook should be launched from a session of jupyter notebook that was launched from a DB2 command window
To do this, run an administrator DB2 command window as an administrator and type 'jupyter notebook'

Set up the enviornment:

In [2]:
import sys,os,os.path
os.environ['IBM_DB_HOME']='C:\Program Files\IBM\SQLLIB'
!pip install ipython-sql
!pip install ibm_db 
!pip install ibm_db_sa

Restart the Kernel if this is your first time installing the above. The next steps will fail unless you do this.

Import the modules and load the SQL magic

In [3]:
import ibm_db
import ibm_db_sa
import sqlalchemy
%load_ext sql
from IPython.display import HTML

In [23]:
## Hide code cells
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

Connect to the database. Change the values of user, host, and password to match your environment. For connection to a local host, use 'localhost' for the host name. Also change the port number and database name in the connection string.

In [4]:
user='db2admin'
host='localhost'
# Define filename for passwords
filename = 'ember_variables.py'
# source the file
%run $filename
password = LocalDB2password
port=50000
db='SAMPLE'
%sql db2+ibm_db://$user:$password@$host:$port/$db

'Connected: db2admin@SAMPLE'

# Db2 Snapshot
Ember's snapshot for Db2 (LUW)

## Instance Uptime

In [7]:
!db2pd -


Database Member 0 -- Active -- Up 4 days 02:04:38 -- Date 2017-07-28-09.25.16.250000



## Database active since

In [8]:
%sql select db_conn_time from table(mon_get_database(-2)) as mgd

Done.


db_conn_time
2017-07-27 18:39:16


## Five most recent activation times

In [9]:
## Not working, come back to it
%sql select * \
from table(pd_get_log_msgs(current timestamp - 7 days, -2))\
where DBNAME= :db 


Done.


TIMESTAMP,timezone,instancename,DBPARTITIONNUM,dbname,pid,processname,tid,appl_id,component,FUNCTION,probe,msgnum,msgtype,msgseverity,msg,MEMBER


## Last Succesful Backup

In [10]:
%sql with ts as (select EID from sysibmadm.db_history where start_time = (select max(timestamp(start_time)) as ts  \
		from sysibmadm.db_history \
		where operation='B' and SQLCODE is NULL)\
	) \
select case \
		when operationtype = 'D' then 'DELTA_OFFLINE'\
		when operationtype = 'E' then 'DELTA_ONLINE' \
		when operationtype = 'F' then 'OFFLINE' \
		when operationtype = 'I' then 'INCR_OFFLINE' \
		when operationtype = 'N' then 'ONLINE' \
		when operationtype = 'O' then 'INCR_ONLINE' \
		else operationtype \
	end as type \
        , to_char(timestamp(start_time), 'YYYY-MM-DD HH24:MI:SS')  as backup_time \
FROM sysibmadm.db_history dh \
	join ts on dh.eid = ts.eid \
ORDER BY start_time desc \
fetch first 1 row only

Done.


TYPE,backup_time
ONLINE,2017-04-24 13:01:13


## HADR details

In [11]:
#untested - test on HADR database
%sql select hadr_role \
    , hadr_state \
    , hadr_connect_status \
    , hadr_log_gap \
FROM table(mon_get_hadr(-2))

Done.


hadr_role,hadr_state,hadr_connect_status,hadr_log_gap


## Tablespaces

In [12]:
%sql select count(*) as NUM_TBSPS_NOT_AUTORESIZE \
FROM table(mon_get_tablespace(NULL,-2)) \
WHERE TBSP_AUTO_RESIZE_ENABLED=0 \
    AND TBSP_TYPE='DMS'

Done.


num_tbsps_not_autoresize
0


In [13]:
%sql select tbsp_name \
    , decimal(float(tbsp_used_pages)/float(tbsp_usable_pages),5,2) as pct_full \
FROM table(mon_get_tablespace(NULL,-2)) \
WHERE TBSP_TYPE='DMS' \
    AND (TBSP_AUTO_RESIZE_ENABLED=0 OR TBSP_LAST_RESIZE_FAILED=1)

Done.


tbsp_name,pct_full


## Index Read Efficiency

In [14]:
%sql select decimal(float(rows_read)/float(rows_returned),10,0) read_eff \
from table(mon_get_database(-2)) as mgd

Done.


read_eff
20


## Buffer Pool Hit Ratios

In [15]:
%sql select bpu.bp_name \
    , (pool_data_l_reads + pool_xda_l_reads + pool_index_l_reads) as total_l_reads \
    , data_hit_ratio_percent \
    , index_hit_ratio_percent \
    , xda_hit_ratio_percent \
    , col_hit_ratio_percent \
from sysibmadm.mon_bp_utilization bpu join table(mon_get_bufferpool(NULL,-2)) mgbp \
    on bpu.bp_name=mgbp.bp_name \
WHERE bpu.bp_name not like 'IBMSYS%'

Done.


bp_name,total_l_reads,data_hit_ratio_percent,index_hit_ratio_percent,xda_hit_ratio_percent,col_hit_ratio_percent
IBMDEFAULTBP,30061,98.29,95.11,None,None


## Package Cache

In [16]:
%sql select decimal((1-(float(pkg_cache_inserts)/float(pkg_cache_lookups)))*100,5,2) as pkg_cache_hitratio \
    , pkg_cache_num_overflows \
from table(mon_get_database(-2)) as mgd

Done.


pkg_cache_hitratio,pkg_cache_num_overflows
90.64,4


## Catalog Cache

In [17]:
%sql select decimal((1-(float(cat_cache_inserts)/float(cat_cache_lookups)))*100,5,2) as cat_cache_hitratio \
    , cat_cache_overflows \
from table(mon_get_database(-2)) as mgd

Done.


cat_cache_hitratio,cat_cache_overflows
98.74,0


## Transaction Logs

In [18]:
%sql select decimal(100*(float(total_log_used)/float(total_log_available)),5,2) as tot_log_use_pct \
    , sec_logs_allocated \
    , sec_log_used_top \
    , log_reads \
    , log_writes \
from table(mon_get_transaction_log(-2)) as mgtl

Done.


tot_log_use_pct,sec_logs_allocated,sec_log_used_top,log_reads,log_writes
2.01,0,0,0,1064


In [19]:
%sql with t1 as (select substr(start_time, 1, 10) as ts \
  ,count(*) as archives \
from sysibmadm.db_history \
where operation = 'X' \
   and start_time > current timestamp - 7 days \
group by substr(start_time, 1, 10) ) \
select avg(archives) as avg_arch_per_hour \
    , max(archives) as max_arch_in_hour \
    , min(archives) as min_arch_in_hour \
from t1 \

Done.


avg_arch_per_hour,max_arch_in_hour,min_arch_in_hour
2,2,2


## Locking

In [20]:
%sql select deadlocks \
    , lock_timeouts \
    , lock_escals \
from table(mon_get_database(-2)) as mgd

Done.


deadlocks,lock_timeouts,lock_escals
0,0,0


## Sorts

In [21]:
%sql select decimal(float(sort_overflows)/float(total_sorts),5,2) as sort_overflow_pct \
from table(mon_get_database(-2)) as mgd

Done.


sort_overflow_pct
0.00
